---
title: "Event Studies"
subtitle: "Simple Panel Data Approaches with Binary Treatment"
author: Vladislav Morozov  
format:
  revealjs:
    include-in-header: 
      text: |
        <meta name="description" content="Panel data event studies: causal analysis, regression interpretation, and applications to Silicon Valley Bank collapse (lecture notes slides)."/>
    width: 1150
    slide-number: true
    sc-sb-title: true
    incremental: true   
    logo: ../../themes/favicon.ico
    footer: "Panel Data: Event Studies"
    footer-logo-link: "https://vladislav-morozov.github.io/econometrics-2/"
    theme: ../../themes/slides_theme.scss
    toc: TRUE
    toc-depth: 2
    toc-title: Contents
    transition: convex
    transition-speed: fast
slide-level: 4
title-slide-attributes:
    data-background-color: "#045D5D"
    data-footer: " "
filters:
  - reveal-header  
include-in-header: ../../themes/mathjax.html 
highlight-style: tango
open-graph:
    description: "Panel data event studies: causal analysis, regression interpretation, and applications to Silicon Valley Bank collapse (lecture notes slides)."
---




## Introduction {background="#00100F"}
  
### Lecture Info {background="#43464B" visibility="uncounted"}


#### Learning Outcomes

This lecture is about a simple kind of causal studies with panel data — comparing outcomes from before and after treatment

<br>

By the end, you should be able to

- Write down event study estimators with two or more periods of data
- Define an appropriate causal framework
- Prove consistency of event study estimators under an assumption of no trends 
  
#### References
 

::: {.nonincremental}


- Chapter 17 in @Huntington-Klein2025EffectIntroductionResearch: <https://www.theeffectbook.net/ch-EventStudies.html>
- @MacKinlay1997EventStudiesEconomics, @Freyaldenhoven2021VisualizationIdentificationEstimation, @Miller2023IntroductoryGuideEvent
- NBER 2023 Methods Lectures on Linear Panel Event Studies: <https://www.nber.org/conferences/si-2023-methods-lectures-linear-panel-event-studies>
  
:::  


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import statsmodels.api as sm
import yfinance as yf

from pathlib import Path


plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Arial"]

BG_COLOR = "whitesmoke"

### Empirical Motivation {background="#43464B" visibility="uncounted"}

#### Event {#sec-event-study-svb-example}

<div class="rounded-box">

*March 10, 2023*: third-largest bank failure in the US: the Silicon Valley Bank (SVB) collapses


</div>

<br>

- Bank collapses are painful for depositors and creditors
- Collapses also raise fears of broader financial contagion — danger to other financial institutions

#### Empirical Question {.scrollable}

<div class="rounded-box">

How did the collapse of the SVB affect stock prices of other US financial institutions in the next 10 days? 


</div>

- Causal question: comparing prices after collapse with prices without collapse
- How to do this comparison? What assumptions do we need?


In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Expand for list of institutions of interest"

tickers = [
    "ALLY",  # Ally Financial Inc.
    "AMTB",  # Amerant Bancorp Inc.
    "ABCB",  # Ameris Bancorp
    "ASB",   # Associated Banc-Corp
    "AUB",   # Atlantic Union Bankshares Corporation
    "AX",    # Axos Financial Inc.
    "BANC",  # Banc of California Inc.
    "BK",    # Bank of New York Mellon Corporation
    "BAC",   # Bank of America Corporation
    "BOH",   # Bank of Hawaii Corporation
    "BKU",   # BankUnited Inc.
    "BHB",   # Bar Harbor Bankshares Inc.
    "BHLB",  # Berkshire Hills Bancorp Inc.
    "BRBS",  # Blue Ridge Bankshares Inc.
    "CADE",  # Cadence Bank
    "COF",   # Capital One Financial Corporation
    "C",     # Citigroup Inc.
    "CFG",   # Citizens Financial Group Inc.
    "CMA",   # Comerica Incorporated
    "CFR",   # Cullen/Frost Bankers Inc. 
    "FNB",   # F.N.B. Corporation
    "FBK",   # FB Financial Corporation
    "FITB",  # Fifth Third Bancorp
    "HBAN",  # Huntington Bancshares Incorporated
    "KEY",   # KeyCorp
    "MTB",   # M&T Bank Corporation
    "PNC",   # PNC Financial Services Group Inc.
    "RF",    # Regions Financial Corporation
    "STT",   # State Street Corporation
    "SYF",   # Synchrony Financial
    "USB",   # U.S. Bancorp
    "WFC",   # Wells Fargo & Company
    "SCHW",  # Charles Schwab Corporation
    "AXP",   # American Express Company
    "DFS",   # Discover Financial Services
    "NTB",   # Bank of N.T. Butterfield & Son Limited
    "EWBC",  # East West Bancorp Inc.
    "WAL",   # Western Alliance Bancorporation
    "SSB",   # SouthState Corporation
    "WBS",   # Webster Financial Corporation
    "FHN",   # First Horizon Corporation
    "PNFP",  # Pinnacle Financial Partners Inc.
    "HOMB",  # Home BancShares Inc.
    "HTH",   # Hilltop Holdings Inc.
    "GBCI",  # Glacier Bancorp Inc.
    "BOKF",  # BOK Financial Corporation
    "ZION",  # Zions Bancorporation
    "TCBI",  # Texas Capital Bancshares Inc.
    "CIVB",  # Civista Bancshares Inc.
    "CFFI",  # C&F Financial Corporation
    "BANF",  # BancFirst Corporation
    "FULT",  # Fulton Financial Corporation
    "ONB",   # Old National Bancorp
    "PB",    # Prosperity Bancshares Inc.
    "UBSI",  # United Bankshares Inc.
    "VLY",   # Valley National Bancorp
    "TRMK",  # Trustmark Corporation 
    "CASH",  # Meta Financial Group Inc.
    "CUBI",  # Customers Bancorp Inc.
    "CFFN",  # Capitol Federal Financial Inc.
    "FFIN",  # First Financial Bankshares Inc. 
    "SFBS",  # ServisFirst Bancshares Inc.
    "TBBK",  # The Bancorp Inc. 
    "WSBC",  # WesBanco Inc.
    "WTFC",  # Wintrust Financial Corporation
]

::: footer

:::

## Two Periods {background="#00100F"}
 
 
### Estimator {background="#43464B" visibility="uncounted"}


#### Simple Event Study Setting

Begin with the simplest possible panel setting with binary treatment: 
 
- Two periods with $N$ units:
  - No treatment in period 1
  - All units treated in period 2
- Data: outcomes $(Y_{i1}, Y_{i2})$.

. . .

Object of interest: "average effect of treatment"

#### Simple Estimator — Average Change

Simplest approach: compute average change in $Y_{it}$ across periods
  $$
  \widehat{AE}_{ES} = \dfrac{1}{N}\sum_{i=1}^N (Y_{i2}- Y_{i1}).
  $$ {#eq-causal-event-studies-naive-panel-contrast}
  

Estimator ([-@eq-causal-event-studies-naive-panel-contrast]) — simplest example of *event study* estimators [see  @Freyaldenhoven2021VisualizationIdentificationEstimation;@Miller2023IntroductoryGuideEvent]

#### Example Framework {#sec-event-study-example}

Possible empirical framework

- Units $i$: firms that make phones
- Outcome $Y_{it}$: their stock price
- Periods:
  1. One week before Apple announces the iPhone
  2. One week after the announcement

. . .

Effect of interest: change in stock prices due to the announcement of iPhone




#### What Does ([-@eq-causal-event-studies-naive-panel-contrast]) Do?


<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-limit}

## Asymptotics for $\widehat{AE}_{ES}$

Let 

::: {.nonincremental}
- *(Cross-sectional random sampling)*: $(Y_{i1}, Y_{i2})$ be independent and identically distributed (IID)
- *Finite first moments*: $\E[\abs{Y_{it}}]<\infty$
:::
 
Then
$$
\widehat{AE}_{ES} \xrightarrow{p} \E[Y_{i2} - Y_{i1}].
$$ 
:::

</div>

### Causal Analysis {background="#43464B" visibility="uncounted"}


#### Causal Framework


Is $\E[Y_{i2} - Y_{i1}]$ interesting (=causal)?

. . .

::: {.callout-important appearance="minimal"}

Need a causal framework to talk about causal effects!

:::

. . .

Again work in the familiar potential outcomes framework: 

- $Y_{it}^0$ — outcome for $i$ in period $t$ if *not treated*
- $Y_{i1}^1$ — outcome for $i$ in period $t$ if *treated*
- Treatment effect for $i$ in $t$: $Y_{it}^1- Y_{it}^0$

. . .

For short, use $Y_{it}^d$ where $d=0, 1$



#### Limit of ES Estimator and Causality

Potential and *realized* outcomes are connected as
$$
Y_{i2} = Y_{i2}^1, \quad Y_{i1} = Y_{i1}^0.
$$


. . .

It follows that
$$
\widehat{AE}_{ES} \xrightarrow{p} \E[Y_{i2}^1- Y_{i1}^0].
$$

::: {.callout-important appearance="minimal"}
 
$\E[Y_{i2}^1- Y_{i1}^0]$ is not necessarily a treatment effect — mixes effect of treatment and effects of time!
::: 

#### Example

[[Context]{.button}](#sec-event-study-example) Again consider the iPhone example. Then 

- $Y_{i2}^1 - Y_{i2}^0$ — treatment effect, change in price *because* of the iPhone announcement
- $Y_{i2}^0 - Y_{i1}^0$ — change in a world *without* iPhone
  
. . . 

<br>

Realized difference = combination of both changes
$$
Y_{i2} - Y_{i1} = Y_{i2}^1- Y_{i1}^0 = [Y_{i2}^1- Y_{i2}^0] + [Y_{i2}^0 - Y_{i1}^0]
$$



#### Solution: Restrict Changes over Time

Simple solution: rule out changes over time

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

**Assumption**: no variation in potential outcomes
$$
Y_{i2}^d= Y_{i1}^d, \quad d=0, 1
$$ 
 

</div>

Then $\widehat{AE}_{ES}$ is estimating a *causal* parameter — average effects:
$$
\begin{aligned}
\widehat{AE}_{ES} & \xrightarrow{p} \E[Y_{i1}^1- Y_{i1}^0]  =  \E[Y_{i2}^1- Y_{i2}^0]
\end{aligned}
$$ 

#### Relaxing Time Invariance: No Trends

Time invariance very strict. Why use it if we only work with averages?

. . .

Weaker assumption:

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

**Assumption** (*no trends*):  
$$
\E[Y_{i2}^d] = \E[Y_{i1}^d], \quad d=0, 1
$$
 

</div>
Allows random variation in potential outcomes between times

#### Summary so Far


<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-no-trends}

## Causal asymptotics for $\widehat{AE}_{ES}$

Let 

::: {.nonincremental}
- Assumptions of @prp-event-study-limit hold
- Assumption of no trends holds
:::
 
Then $\widehat{AE}_{ES}$ consistent for causal parameters:
$$
\widehat{AE}_{ES} \xrightarrow{p} \E[Y_{i1}^1- Y_{i1}^0]  =  \E[Y_{i2}^1- Y_{i2}^0]
$$ 
:::


</div>

### Regression Interpretation {background="#43464B" visibility="uncounted"}

 

#### Regression Setting

Can also connect $\widehat{AE}_{ES}$ and OLS 

. . . 
 
Consider regressing $Y_{it}$ on $(1, D_{it})$ where
$$
\begin{aligned}
% Y_{it} & = \beta_0 + \beta_1 D_{it} + U_{it}, \\
D_{it} & = \begin{cases}
1, & t= 1 \\
0, & t =0
\end{cases}
\end{aligned}
$${#eq-event-study-simple-regression}
and where we simply treat $(Y_{i1}, D_{i1})$ and $(Y_{i2}, D_{i2})$ as separate observations



#### Event Studies and OLS

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-no-trends}

## $\widehat{AE}_{ES}$ is OLS

For $\beta_1$ of regression  ([-@eq-event-study-simple-regression])
$$
\widehat{AE}_{ES} = \hat{\beta}_1,
$$ 
where $\hat{\beta}_1$ is the estimator of the coefficient on $D_{it}$
:::


</div> 

Can use all results developed for OLS for $\widehat{AE}_{ES}$
 
   

#### Event Study and Regression I

A way to think about regression in causal settings:

- Write down the regression in terms of parameters of interest: e.g. let
$$ 
\begin{aligned}
Y_{it} & = \beta_0 + \beta_1 D_{it} + U_{it},
\\
\beta_0 & = \E[Y_{i1}^0], \quad
\beta_1  =  \E[Y_{i2}^1- Y_{i2}^0] 
\end{aligned}
$$
- Connect regression to potential outcomes: *what is $U_{it}$ in terms of potential outcomes? (exercise)* 
 
#### Event Study and Regression II

Let $\bX_{it} = (1, D_{it})'$. Then

- By properties of OLS know that OLS is consistent for $\bbeta = (\beta_0, \beta_1)$ if
  -  $\E[\bX_{it}U_{it}] =0$  
  -  $\E[\bX_{it}\bX_{it}']$ invertible

- So just need to check if this $U_{it}$ satisfies $\E[\bX_{it}U_{it}] =0$ 
- If yes, OLS can estimate average effects of interest
 

. . .

::: {.callout-important appearance="minimal"}

Remember: the OLS "model" and the underlying causal model are separate things! Here the causal model is "nonparametric"
:::


::: footer

:::




## Multiple Periods {background="#00100F"}

 
### Estimation and Causal Framework {background="#43464B" visibility="uncounted"}

 
#### Multiple Period Framework

- Often have more than 2 periods of data
- Want to use that data

. . . 

New framework:

- $T$ periods in total
- Treatment starts in period $t_0$
- We see $Y_{it}^0$ for $t<t_0$ and $Y_{it}^1$ for $t\geq t_0$

#### Expanded Regression

New variables for treatment: 
$$\small
D_{it, \tau} = \begin{cases}
1, & t= \tau, \\
0, & t\neq \tau
\end{cases}
$$
 
. . . 

Can try similar regression:
$$\small
Y_{it} = \beta_0 +  \sum_{\tau = t_0}^{T} \beta_\tau D_{it, \tau} + u_{it}
$${#eq-event-studies-long-regression}
Let $\hat{\bbeta}=(\hat{\beta}_0, \hat{\beta}_{t_0}, \dots, \hat{\beta}_{T}$) be the OLS estimator

#### OLS Estimator Expression 

Fairly easy to show that
$$
\begin{aligned}
\hat{\beta}_{\tau} & = \dfrac{1}{N} \sum_{i=1}^N Y_{i\tau} - \dfrac{1}{N(t_0-1)} \sum_{i=1}^N\left[ Y_{i1} + \dots + Y_{it_0-1} \right] \\
& \xrightarrow{p} \E\left[Y_{i\tau}^1  - \dfrac{1}{t_0-1}(Y_{i1}^0+ \dots + Y_{it_0-1}^0) \right]
\end{aligned}
$$
More general version of the simple estimator of before

#### Dynamic Treatment Effects?

If $\beta_{\tau}$ — average effect in period $\tau$, then model ([-@eq-event-studies-long-regression]) seems to allow for *dynamic* effects


. . . 
 
<br>

Dynamic effects <span class="highlight">realistic</span>: effect of treatment may change over time. Example: impact of job training on earnings:

- Disappearing: you forget the training over time
- Increasing: job training lets you jump to a higher position and gain experience quicker for the rest of your life

 
#### No Trends Rule Out Dynamics

Suppose that the *no trends* assumption holds
$$
\begin{aligned}
& \E\left[Y_{i\tau}^1  - \dfrac{1}{t_0-1}(Y_{i1}^0+ \dots + Y_{it_0-1}^0) \right]  = \E[Y_{it}^1 - Y_{it}^0]
\end{aligned}
$$
The right hand side does <span class="highlight">not</span> depend on $t$

. . . 

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

No dynamic treatment effects under no trends!

</div> 
<span class="highlight">Bad</span>, since we want to allow for dynamics


#### Relaxing No Trends Assumption

Can relax no trends to only affect *one* of the potential outcomes — makes more sense to restrict the untreated outcome (why?)

<br>

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

**Assumption** (*no trends in the baseline*): $\E[Y_{it}^0]$ does not depend on $t$
 

</div>
 
#### Recovering Dynamic Average Treatment Effects


Under assumption of no trends in the baseline:
$$
\begin{aligned}
& \E\left[Y_{i\tau}^1  - \dfrac{1}{t_01}(Y_{i1}^0+ \dots + Y_{it_0-1}^0) \right]  = \E[Y_{i \tau}^1 - Y_{i\tau}^0]
\end{aligned}
$$

<br> 

Right hand side is average effect in period $\tau$

 

### Asymptotic Properties {background="#43464B" visibility="uncounted"}


#### Consistency in the Multivariate Case

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-no-trends-multivariate}

Let 

::: {.nonincremental}
- Assumptions of @prp-event-study-limit hold
- No trends in the baseline assumption hold
:::
 
Then $\hat{\beta}_{\tau}$ consistently estimates the average treatment effect in period $\tau$ given by $\E[Y_{i \tau}^1 - Y_{i\tau}^0]$. 
:::




</div>

#### Unbiasedness of OLS

Moreover, under no trends in the baseline
$$
\E[\hat{\beta}_{\tau}] = \E[Y_{i \tau}^1 - Y_{i\tau}^0]
$$

<br>

. . .


In other words, the OLS estimator is unbiased

#### Asymptotic Distribution

 
<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-no-trends-multivariate-distribution}

Let 

::: {.nonincremental} 
- No trends in the baseline assumption hold 
- $(Y_{i1}, Y_{i2}, \dots, Y_{iT})$ be IID (over $i$)
- *Finite second moments*: $\E[Y_{it}^2]<\infty$
:::
 
Then there exists some variance $V$ such that
$$
\sqrt{N}\left( \hat{\beta}_{\tau} - \E[Y_{i \tau}^1 - Y_{i\tau}^0] \right) \Rightarrow N(0, V)
$$
:::


</div>

<!-- #### Asymptotic Variance I

Need variance $V$ to construct confidence intervals for $\E[Y_{i \tau}^1 - Y_{i\tau}^0]$. How to find $V$? Remember the CLT:

. . .

<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

If $X_1, X_2, \dots$ are IID random variables with $\E[X_i]=\mu$ and $\E[X^2]<\infty$, then
$$
\sqrt{N}\left( \dfrac{1}{N}\sum_{i=1}^N X_i - \mu  \right)\Rightarrow N\left(0, \var(X_i) \right)
$$
Asymptotic variance = variance of $X_i$
</div>
  -->

####  Asymptotic Variance 

Need variance $V$ to construct confidence intervals for $\E[Y_{i \tau}^1 - Y_{i\tau}^0]$. 

. . . 

$$
\begin{aligned}
\hat{\beta}_{\tau} & = \dfrac{1}{N} \sum_{i=1}^N Z_i\\
Z_i & =  Y_{i\tau} - \dfrac{1}{(t_0-1)}\left[ Y_{i1} + \dots + Y_{it_0-1} \right]
\end{aligned}
$$

. . .
 
From central limit theorem:
$$
V = \var(Z_i)
$$

#### Estimator for Asymptotic Variance

Can estimate $V$ with 
$$
\hat{V} = \widehat{\var}(Z_i) = \dfrac{1}{N}\left(Z_i - \dfrac{1}{N}\sum_{j=1}^N Z_j \right)^2
$$

. . . 

Estimated standard error of $\hat{\beta}_{\tau}$:
$$
\widehat{se}(\hat{\beta}_{\tau}) = \sqrt{ \dfrac{\hat{V}}{N} }
$$

#### Inference on Average Effects I

Can now construct confidence intervals and hypothesis tests about $\E[Y_{i \tau}^1 - Y_{i\tau}^0]$. E.g. an asymptotic 95% confidece interval:
$$
\widehat{CI}_{95\%} = \left[\hat{\beta}^{OLS}_{\tau} - z_{1-\alpha/2}\widehat{se}(\hat{\beta}_{\tau}), \hat{\beta}^{OLS}_{\tau} + z_{1-\alpha/2}\widehat{se}(\hat{\beta}_{\tau}) \right]
$$
where the critical values $z_{1-\alpha/2}$ come from the standard normal distribution
$$
 z_{1-\alpha/2}= \Phi^{-1}\left(1 - \dfrac{\alpha}{2} \right)
$$

#### Inference on Average Effects II

But what if we want to set the *joint* hypothesis
$$
H_0: \beta_{\tau} = 0, \quad \tau = t_0, \dots, T
$$

. . .

- This $H_0$ is a hypothesis above a *vector* of effects 
- To write down a Wald test we need the *joint* asymptotic distribution of $\hat{\bbeta}$


#### Joint Asymptotic Distribution


<div style="border: 2px solid #ccc; padding: 9px; border-radius: 15px; margin-bottom: 10px;">

::: {#prp-event-study-joint-limit-distribution}

## Joint Asymptotics for Estimated Effects

Let 

::: {.nonincremental} 
- Assumption of no trends in the baseline hold
- $(Y_{i1}, Y_{i2}, \dots, Y_{iT})$ be IID (over $i$) with finite second moments
:::
 
Then  
$$ \small
\sqrt{N}(\hat{\bbeta} -\bbeta) \Rightarrow N(0, \avar(\hat{\bbeta}))
$$ 
:::

</div>

#### Joint Inference on Average Effects: Wald Test

Can use @prp-event-study-joint-limit-distribution to create a Wald test:

- Write $H_0$ as $H_0: \bR\bbeta = \bq$ for
$$ \small
\bR = \begin{pmatrix}
\mathbf{0} & \bI_{T-t_0+1}
\end{pmatrix}, \quad \bq = \mathbf{0}
$$

- Write down Wald statistic:
$$ \small
W = N(\bR\hat{\bbeta}-\bq)'(\bR\widehat{\avar}(\hat{\bbeta})\bR')^{-1}(\bR\hat{\bbeta}-\bq)
$$

- Compare to $(1-\alpha)$th quantile of $\chi^2_{T-t_0+1}$ distribution

## Empirical Application {background="#00100F"}


### Formalizing the Application {background="#43464B" visibility="uncounted"}

#### Back to Empirical Application

Now let's answer our empirical question — impact of the SVB collapse on stock returns of US financial institutions [[Context]{.button}](#sec-event-study-svb-example)

<br>

. . . 

We need

- Outcome variables (which returns?)
- Time context $t$
- How to define the treatment 

#### Some Finance Background: Abnormal Returns 

In finance, usually work with <span class="highlight">abnormal returns $AR_{it}$</span> [@MacKinlay1997EventStudiesEconomics]

<br>

- Each stock $i$ assumed to have a "normal" or "expected" return $ER_{it}$ given market conditions on day $t$ — everything "expected" by the market
- $AR_{it}$ — differences between actual return $R_{it}$ and $ER_{it}$:
$$ \small
R_{it} = ER_{it} + AR_{it}
$$

#### Interest: Impact on Abnormal Returns

Outcome of interest: $AR_{it}$

<br>

. . .
 

|$d=0$ | $d=1$|
|:---:|:----:|
|Only idiosyncratic day-to-day shifts| Impact of SVB collapse + idiosyncratic movements|

: Potential outcomes {.striped .hover .bordered}

<br> 

<div class="rounded-box">

Assume that $\E[AR_{it}^0] = 0$ and $AR_{it}^0$ uncorrelated with broader market characteristics

</div>  

::: footer


:::

#### Computing Expected and Abnormal Returns

Expected returns usually given by some model:

- Simple mean model: let $R_{it}$ be return of asset $i$ on $t$. Expected return $ER_{it} = \E[R_{it}]$
- Factor models: 
$$\small
ER_{it} = f_i\left( \text{Some market characteristics on }t \right)
$$
Example: $ER_{it} = \bbeta_i'\bx_{t}$ where $\bx_{t}$ includes market return on $t$ (CAPM), or also small minus big, high minus low (Fama-French 3 factor)

::: footer

Check out @Fama2014TwoPillarsAsset Nobel Prize lecture for some background and history

:::

#### Estimating Model Parameters

- Can compute $ER_{it}$ (and $AR_{it}$) if know model parameters
- But how to compute parameters? 

. . . 

<br> 

Select estimation window with $D_{it}=0$. Then 
$$
R_{it} = \bbeta_i'\bx_t + AR_{it}^0, \quad \E[AR_{it}^0\bx_t] =0
$$
Can consistently estimate $\bbeta_i$ by regressing $R_{it}$ on $\bx_t$ (why — measurement error in dependent variable)

#### Computing Abnormal Returns

Work with estimated abnormal returns:
$$
\widehat{AR}_{it} = R_{it} - \widehat{ER}_{it} = R_{it} - \hat{\bbeta}'\bx_t
$$

<br>

One issue: $\widehat{AR}_{it}$ has measurement error. But

- Measurement error in dependent variable not a problem if uncorrelated with covariates
- If estimation window for $\hat{\bbeta}$ large, measurement error likely small

### Preparing Data {background="#43464B" visibility="uncounted"}

#### Define Time Frames

When does treatment turn on? What $T$ to take? 

- Key public announcement — March 8, 2023 (our $t_0$)
- Look for 10 days around ($T=20$)

. . .


In [ ]:
#| echo: true
event_date = pd.Timestamp("2023-03-08")
event_window = pd.Timedelta(days=10)  

<br> 

. . . 

For estimation, use long window before SVB collapse:


In [ ]:
#| echo: true
estimation_window = pd.Timedelta(days=500) 
start_date = event_date - event_window - estimation_window 

#### Obtaining Data {.scrollable}

Necessary data:

- Returns on tickers of interest
- Market data for estimating $ER_{it}$ — will use 3-factor Fama-French model

. . .

Can obtain ticker data from Yahoo Finance directly with  `yfinance` package (abbreviated as `yf`)


In [ ]:
#| echo: true
#| output: false
#| code-fold: true
#| code-summary: "Expand for data call and data preparation"

# Download the tickers and rename columns to only retain ticker names
stock_data = yf.download(
    tickers, 
    start=start_date, 
    end=event_date + 4*event_window,
    progress=False,
)
stock_data = stock_data.iloc[:, 0:len(tickers)]
stock_data.columns = (
    stock_data.columns.map(lambda col: re.sub(r"[()\s']", "", col[1]))
)

# Calculate daily returns
returns = stock_data.pct_change().dropna()*100

# Read in FF 3 factor daily data
ff_path = Path() / 'data' / 'fama-french-3.csv'
fama_french_data = pd.read_csv(ff_path).iloc[:, :-1]
# Read the Date column as a date and set it as the index
fama_french_data["Date"] = pd.to_datetime(
    fama_french_data["Date"], 
    format="%Y-%m-%d", 
)
fama_french_data = fama_french_data.set_index("Date")

# Merge returns and FF data, drop unnecessary data
merged_data = pd.concat(
    [returns, fama_french_data], 
    axis=1
).dropna(axis=0)

::: footer

:::

#### Estimating Abnormal Returns {.scrollable}

Estimate outcome variable $AR_{it}$ using Fama-French 3 market model:

- Separately regress returns of each $i$ on Fama-French factors over estimation window
- Obtained fitted values ($\widehat{ER}_{it}$) 
- Compute $\widehat{AR}_{it}$} and store

For contrast, also estimate mean model


In [ ]:
#| echo: true
#| output: false
#| code-fold: true
#| code-summary: "Expand for estimation of abnormal returns"

# Create arrays to hold abnormal returns   
abnormal_returns_factor = pd.DataFrame()
abnormal_returns_mean = pd.DataFrame()    

# Estimate abnormal returns for each 
for ticker in tickers:
    # Estimate the 3-factor abnormal returns
    X = merged_data.loc[
        merged_data.index < event_date - event_window, 
        ["Mkt-RF", "SMB", "HML"]
    ]   
    y = merged_data.loc[
        merged_data.index < event_date - event_window, 
        ticker
    ] 
    X = sm.add_constant(X) 
    model = sm.OLS(y, X).fit()   

    # Compute expected returns during the window
    X_event = (
        merged_data.loc[
            ((merged_data.index >= event_date - event_window) & 
            (merged_data.index <= event_date + event_window)), 
            ["Mkt-RF", "SMB", "HML"]
        ]
    )
    X_event = sm.add_constant(X_event)
    expected_returns = model.predict(X_event)

    # Extract realized returns during the event
    event_data = (
        merged_data.loc[
            ((merged_data.index >= event_date - event_window) & 
            (merged_data.index <= event_date + event_window)), 
            ticker
        ]
    )

    # Abnormal returns are residuals
    abnormal_returns_factor[ticker] = event_data - expected_returns
    abnormal_returns_mean[ticker] = event_data - y.mean()

::: footer

:::

#### Visualizing Average Abnormal Returns


In [ ]:
#| label: fig-causal-es-avg-abnormal-return
#| fig-cap: Averages of abnormal returns with 95% pointwise confidence intervals

# Combine the two DFs of abnormal returns into a list
abnormal_returns_list = [abnormal_returns_mean, abnormal_returns_factor] 
abnormal_returns_titles = ["Expected returns: own average", "Expected returns: three-factor model"]

# Create a figure
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 4.5))
fig.patch.set_facecolor(BG_COLOR)
fig.patch.set_edgecolor("teal")
fig.patch.set_linewidth(5)

# Add plots of average abnormal returns + CIs
for method_id, abnormal_df in enumerate(abnormal_returns_list):
    # Compute averages and bounds for the confidence interval
    effects = pd.DataFrame()
    effects["estimates"] = abnormal_df.cumsum().mean(axis=1)
    effects["ci_upper"] = effects["estimates"] + 1.96 * np.sqrt(
        abnormal_df.var(axis=1).cumsum()
    )
    effects["ci_lower"] = effects["estimates"] - 1.96 * np.sqrt(
        abnormal_df.var(axis=1).cumsum()
    )

    # Plot the estimates
    axs[method_id].plot(
        effects.index,
        effects["estimates"],
        color="blue",
        linewidth=2,
        label="Average abnormal returns",
    )

    # Plot the confidence intervals
    axs[method_id].fill_between(
        effects.index,
        effects["ci_lower"],
        effects["ci_upper"],
        color="lightblue",
        alpha=0.5,
        label="95% CI",
    )

    # Add a vertical line on event date
    vertical_line_date = event_date
    axs[method_id].axvline(
        x=vertical_line_date,
        color="red",
        linestyle="--",
        label="Event Date",
    )

    # Add labels and title
    axs[method_id].set_xlabel("Date")
    axs[method_id].xaxis.set_tick_params(
        which="both",
        rotation=76,
        labelleft=True,
        left=True,
    )
    axs[method_id].set_title(
        abnormal_returns_titles[method_id],
        loc="left",
    )

    # Align y-axes and add grid
    axs[method_id].set_ylim([-48, 18])
    axs[method_id].grid(True)

    axs[method_id].set_facecolor(BG_COLOR)
    if method_id == 1:
        axs[1].legend()

fig.suptitle("Averages of abnormal returns per date", size=18)

plt.show()

::: footer

:::

#### Looking For Pretrends

@fig-causal-es-avg-abnormal-return suggests idea:

- Cannot check no trends assumption for all periods (why?)
- But can check pretreatment periods for trends in means

<br>

. . . 

@fig-causal-es-avg-abnormal-return: zero mean for $t$ before March 8 — supports no trends


### Estimation and Inference {background="#43464B" visibility="uncounted"} 

#### Preparing Data for Event Study

Now need to create $\bX$ and $\by$ matrices to apply event study OLS 

<br>

. . . 

Key challenge: how do $\bX$ and $\by$ look like?

- Regressors: indicators $D_{it, \tau}$ of every day after treatment start 
- Express in tabular form: each row in $\bX$ contains data on one stock $i$ on one day $t$, columns — $\widehat{AR}_{it}$ and $D_{it, \tau}$

#### Generate Event Study Data {.scrollable}


In [ ]:
#| echo: true 
#| code-fold: true
#| code-summary: "Expand for construction of data matrices"

# Create an array for day dummies
dummy_df = (
    pd.get_dummies(
        np.maximum(
            (abnormal_returns_mean.index - pd.to_datetime(event_date)).days+1, 
            0,
        ), 
        prefix='day',
    )
)

# Set the index again
dummy_df.index = abnormal_returns_factor.index
# Drop the column of days preceding the event
dummy_df = dummy_df.drop("day_0", axis=1)

# Melt abnormal returns data
ar_factor_long = abnormal_returns_factor.reset_index().melt(id_vars="Date")
ar_factor_long.head()

# Merge data
ar_factor_days = pd.merge(
    ar_factor_long, 
    dummy_df, 
    how='left', 
    left_on='Date', 
    right_index=True,
)
# Split into two
exog = ar_factor_days.filter(regex="day*", axis=1).astype('float64')
exog = sm.add_constant(exog)
endog = ar_factor_days.loc[:, "value"]

# View
ar_factor_days.head()

::: footer


:::

#### Event Study OLS {.scrollable}

Can now apply the event study estimator 


In [ ]:
#| echo: true

factor_model = sm.OLS(endog, exog).fit(cov_type="HC0")
print(factor_model.summary())

::: footer

:::


#### Cumulative Effects {.scrollable}

There are (at least) two kinds parameters of interest:

- Coefficients $\beta_{\tau}$
- Cumulative effects up to each given day

Can compute cumulative effects and their variances as linear combinations of $\beta_{\tau}$

#### Computing Cumulative Effects {.scrollable}


In [ ]:
#| echo: true 
#| code-fold: true
#| code-summary: "Expand to see computation of cumulative effects"
# Extract coefs and SEs of daily indicators
dummy_coefficients = factor_model.params.iloc[1:]  
dummy_cov_matrix = factor_model.cov_params().iloc[1:, 1:]  

# Compute the cumulative effects
cumulative_effects = dummy_coefficients.cumsum()

# Compute the standard errors of the cumulative effects
cumulative_se = np.zeros(len(cumulative_effects))
for t in range(1, len(cumulative_effects) + 1):
    # Create a vector of ones up to day t and zeros afterwards
    indicator_vector = np.concatenate(
        [np.ones(t), np.zeros(len(cumulative_effects) - t)]
    )

    # Compute the SE of the cumulative effect
    cumulative_se[t - 1] = np.sqrt(
        indicator_vector @ dummy_cov_matrix @ indicator_vector
    )

# Create a DataFrame to store the results
results = pd.DataFrame(
    {
        "Day": factor_model.params.index[1:].map(lambda x: x[4:]),
        "Cumulative Effect": cumulative_effects,
        "SE": cumulative_se,
    }
)

# Prepend day 0 as a reference
day0_df = pd.DataFrame(
    {
        "Day": 0,
        "Cumulative Effect": 0,
        "SE": 0,
    },
    index = np.array(['day_0'])
)
results = pd.concat([day0_df, results])

# Construct confidence intervals
results["ci_upper"] = results["Cumulative Effect"] + 1.96 * results["SE"]
results["ci_lower"] = results["Cumulative Effect"] - 1.96 * results["SE"]

# Set index
results.index = abnormal_returns_mean.index[6:]

# Print the results
print(results)

::: footer

:::


#### Visualizing Cumulative Effects


In [ ]:
# Reset the index, plotting will be done against the new index
results_reset = results.reset_index()

fig, ax = plt.subplots(figsize=(14, 4.5))
fig.patch.set_facecolor(BG_COLOR)
fig.patch.set_edgecolor("teal")
fig.patch.set_linewidth(5)

# Add vertical line before the event: now use 0.5 as potition
ax.axvline(
    x=0.5,
    color="red",
    linestyle="--",
    label="Event Date",
)
ax.axhline(
    y=0,
    color="gray",
    linestyle="--",
    label="_nolegend_",
)

# Plot the estimates as dots
ax.plot(
    results_reset.index,
    results_reset["Cumulative Effect"],
    "o",
    color="blue",
    markersize=8,
    label="Estimates",
)

# Add error bars
ax.errorbar(
    results_reset.index,
    results_reset["Cumulative Effect"],
    yerr=[
        results_reset["Cumulative Effect"] - results_reset["ci_lower"],
        results_reset["ci_upper"] - results_reset["Cumulative Effect"],
    ],
    fmt="o",
    color="blue",
    ecolor="gray",
    capsize=5,
    capthick=2,
    label="95% CI",
)

# Add labels and title
ax.set_xlabel("Date")
ax.set_ylabel("Average Cumulative Effect on Other Stock Returns")
ax.set_title(
    "Event Study: Collapse of the Silicon Valley Bank (3-factor model)",
    loc="left",
    size=14,
)
ax.set_ylim([-10.5, 4])
ax.legend()

# Add tick labels using the original dates
ax.set_xticks(
    results_reset.index,
    results_reset["Date"].dt.strftime("%Y-%m-%d"),
    rotation=45,
)

# Recolor and add grid
plt.grid(True)
ax.set_facecolor(BG_COLOR)

# Show the plot
plt.show()

#### Testing Joint Zero Effect

Can also formally test our null that $H_0: \beta_{\tau}=0$ for all $\tau$

<br>


. . .

Use Wald test: 

In [ ]:
#| echo: true
R = np.concatenate( (np.zeros((8, 1)), np.eye(len(factor_model.params)-1)), axis=1)
wald_results = factor_model.wald_test(R,  scalar=True)
print(wald_results)

#### Empirical Conclusions

So what is the final conclusion?

<br>


<div class="rounded-box">

Find that collapse of the SVB had significant negative effects on returns over the next 10 days

</div>



## Recap and Conclusions {background="#00100F"}
  
#### Recap

In this lecture we

1. Introduced event studies — the simplest panel causal estimator
2. Discussed causal properties under various assumptions of no trends
3. Considered dynamic treatment effects
4. Expressed event studies as regression
5. Did a detailed study on the collapse of the SVB

#### Next Questions

- What if the assumption of no trends is unreasonable?
- What if the treatment is not binary?

#### References {.allowframebreaks visibility="uncounted"}

::: {#refs}
:::